In [1]:
import ehm_dmrg.ehm_dmrg as ehm
import ehm_dmrg
import pandas as pd
from pathlib import Path
import importlib
import ehm_dmrg.dmrg_performance_metrics as dpm
import json
import time


In [2]:
calc_resistance_matrix_properties = True
fcidump_location = "/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts"
json_feature_storage = "/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/feature_store/initial_catalysts_plus_small.json"


#Location of csv files
# datafiles_folder = "/mnt/d/ZapataCalcs/GSEE/DMRG/DMRG_output_data"
datafiles_folder = "/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/performance_metrics_store"
# datafiles_folder = "../"
# datafile_name = "hami_9_data.csv"

# fcidump_files =["fcidump.2_co2_6-311++G__","fcidump.36_1ru_II_2pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }"]
fcidump_files = [
    "fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}",
    "fcidump.10_fecp2+_s0.5_def2-tzvp",
    "fcidump.11_fecp2_s0_def2-tzvp",
    "fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}",
    "fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)",
    "fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)",
    "fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.28_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.29_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.30_4a_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.31_4a_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}",
    "fcidump.3_ts_ru_macho_co2_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.45_1_star_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.49_2_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.40_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.41_1_ts_{'default' _ 'cc-pVTZ', 'Ru' _ 'LANL2TZ'}",
    "fcidump.56_5_{'default' _ '6-31++G(d,p)'}",
    "fcidump.50_6acme_{'default' _ '6-31++G(d,p)'}",
    "fcidump.1_ru_macho_{'Ru'_ 'cc-pVTZ-PP', 'default'_ '6-311++G__'}",
    "fcidump.32_2ru_III_3pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.33_2ru_III_3pl_{'default' _ '6-31+G(d,p)', 'Ru' _ 'lanl2tz' }",
    "fcidump.34_3ruo_IV_2pl_{'Ru' _ 'lanl2tz', 'default' _ '6-31+G(d,p)'}",
    "fcidump.61_3_15_af_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.35_3ruo_IV_2pl_{'Ru' _ 'lanl2tz', 'default' _ '6-31+G(d,p)'}",
    "fcidump.59_5_16_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.63_5_15_af_ts_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.65_5_15_af_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
    "fcidump.60_5_16_{'default' _ 'cc-pVTZ', 'Mn' _ 'lanl2tz'}",
]

In [3]:
# features_dataframe =
fcidump_location = Path(fcidump_location)

importlib.reload(ehm)
importlib.reload(ehm_dmrg)

dmrg_features_molecules_list = []
# load already calculated features
if Path(json_feature_storage).exists():
    with open(json_feature_storage, "r") as file:
        dmrg_features_molecules_list = file.readlines()
    dmrg_features_molecules_list = [json.loads(x) for x in dmrg_features_molecules_list]
    extant_hami_ids = [x["hamiltonian_id"] for x in dmrg_features_molecules_list]

start_time = time.perf_counter_ns()

for fcidump_file in fcidump_files:
    loop_start_time = time.perf_counter_ns()
    fcidump_filepath = fcidump_location / Path(fcidump_file)
    print(fcidump_filepath)

    if fcidump_file in extant_hami_ids:
        print(f"Skipping {fcidump_file} as it already exists in the feature store")
        continue
    
    dmrg_features_one_molecule={}
    dmrg_features_one_molecule["hamiltonian_id"] = fcidump_file

    dmrg_features_one_molecule_new = ehm.dmrg_features_from_fcidump(
        fcidump_filepath=fcidump_filepath,
        MOLPRO_ORBSYM=True,
        calc_resistance_matrix_properties=calc_resistance_matrix_properties,
        verbosity=1,
    )
    dmrg_features_one_molecule.update(dmrg_features_one_molecule_new)
    print(dmrg_features_one_molecule)
    
    with open(json_feature_storage, "a") as file:
            file.write(json.dumps(dmrg_features_one_molecule))  # use `json.loads` to do the reverse
            file.write("\n")
    dmrg_features_molecules_list.append(dmrg_features_one_molecule)
    current_time = time.perf_counter_ns()
    loop_time = current_time - loop_start_time
    
    print(f"Loop time (s) for {fcidump_file} is {loop_time/1e9}")
current_time = time.perf_counter_ns()    
print(f"Total time (s) for all molecules is {(current_time - start_time)/1e9}")

/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}
Skipping fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'} as it already exists in the feature store
/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.10_fecp2+_s0.5_def2-tzvp
Skipping fcidump.10_fecp2+_s0.5_def2-tzvp as it already exists in the feature store
/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.11_fecp2_s0_def2-tzvp
Skipping fcidump.11_fecp2_s0_def2-tzvp as it already exists in the feature store
/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}
Skipping fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'} as it already exists in the feature store
/mnt/d/ZapataCalcs/GSEE/fcidumps_catalysts/fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}
Skipping fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'} as it already ex

In [5]:
ml_features_df = pd.DataFrame(dmrg_features_molecules_list)
ml_features_df

,hamiltonian_id,num_orbitals,num_electrons,num_spin_orbitals,two_S,orbsym,isym,total_num_states,log10_hilbert_space_size,tbt_matrices_calc_time_s,...,density_hopping_tensor_inv_coeff_variation,density_hopping_tensor_sum_abs,tbt_density_hopping_calc_time_s,tbt_mean,tbt_std,tbt_max,tbt_min,tbt_coeff_variation,tbt_inv_coeff_variation,tbt_calc_time_s
0,"fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'...",33,46,66,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.066117e+16,16.609180,0.000129,...,-0.163452,427.329292,0.000234,-0.000181,0.008872,0.267286,-0.419490,-48.890471,-0.020454,0.004729
1,fcidump.10_fecp2+_s0.5_def2-tzvp,46,57,92,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.970221e+25,25.472789,0.000156,...,-0.145057,675.364689,0.012556,-0.000110,0.006374,0.199710,-0.327631,-57.896224,-0.017272,0.625121
2,fcidump.11_fecp2_s0_def2-tzvp,46,58,92,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.792375e+25,25.253429,0.000148,...,-0.145191,643.901094,0.016808,-0.000105,0.006246,0.196475,-0.318972,-59.572611,-0.016786,0.452869
3,"fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'...",33,45,66,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,8.906733e+16,16.949718,0.000099,...,-0.163648,445.456564,0.000227,-0.000208,0.009116,0.267021,-0.459342,-43.897176,-0.022781,0.004100
4,"fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",32,50,64,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.785570e+13,13.679934,0.000068,...,-0.154925,400.407716,0.000229,-0.000267,0.010970,0.185329,-0.572850,-41.013888,-0.024382,0.003799
5,"fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000170,...,-0.124809,861.175857,0.016061,-0.000084,0.005300,0.280269,-0.456726,-63.056685,-0.015859,0.525287
6,"fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",27,44,54,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.393071e+10,10.378956,0.000081,...,-0.170332,325.756966,0.000171,-0.000382,0.013949,0.151781,-0.593228,-36.478773,-0.027413,0.001649
7,"fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000184,...,-0.127855,983.389188,0.020848,-0.000083,0.005330,0.267025,-0.452306,-64.516594,-0.015500,0.465496
8,"fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",32,50,64,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.785570e+13,13.679934,0.000063,...,-0.161113,445.499253,0.000245,-0.000281,0.010940,0.168178,-0.564789,-38.961989,-0.025666,0.004385
9,"fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000185,...,-0.126442,978.289502,0.017098,-0.000081,0.005307,0.248773,-0.445743,-65.850445,-0.015186,0.669791


In [6]:


datafiles_folder_path = Path(datafiles_folder)
# dpm.load_dmrg_output_data(datafile_path=datafiles_folder_path/datafile_name)
csv_file_list = [f"{file}.csv" for file in fcidump_files]
(
        id_list,
        coarse_bond_dimension_list,
        total_loops_cpu_time_sec_list,
        total_loops_wall_time_sec_list,
    ) = dpm.get_coarse_bond_dimension_needed_many_files(
    csv_file_list=csv_file_list, folder_path=datafiles_folder_path, energy_change_threshold_hartrees=1.37e-3
)
print(id_list, coarse_bond_dimension_list)


# performance_metrics_dictionary = [{"hamiltonian_id":id, "coarse_bond_dimension":coarse_bond_dimension, t} for id, coarse_bond_dimension in zip(id_list, coarse_bond_dimension_list)]
performance_metrics_dictionary = [{"hamiltonian_id":id, "coarse_bond_dimension":coarse_bond_dimension, "total_loops_cpu_time_sec":total_loops_cpu_time_sec, "total_loops_wall_time_sec":total_loops_wall_time_sec} for id, coarse_bond_dimension, total_loops_cpu_time_sec, total_loops_wall_time_sec in zip(id_list, coarse_bond_dimension_list, total_loops_cpu_time_sec_list, total_loops_wall_time_sec_list)]
performance_metrics_df = pd.DataFrame(performance_metrics_dictionary)
performance_metrics_df

["fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}", 'fcidump.10_fecp2+_s0.5_def2-tzvp', 'fcidump.11_fecp2_s0_def2-tzvp', "fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'_ 'def2-SVP'}", "fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)", "fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)", "fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.28_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.29_2_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SVP', 'Cl'_ 'def2-SVP', 'default'_ '6-311+G(d,p)'}", "fcidump.30_

,hamiltonian_id,coarse_bond_dimension,total_loops_cpu_time_sec,total_loops_wall_time_sec
0,"fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'...",19,11575.997945,371.488722
1,fcidump.10_fecp2+_s0.5_def2-tzvp,13,33469.678334,1172.941756
2,fcidump.11_fecp2_s0_def2-tzvp,8,14890.202160,538.292357
3,"fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'...",170,52218.561128,1580.383785
4,"fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",9,4534.212040,166.208680
5,"fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",77,164849.615598,5863.330374
6,"fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",9,2490.699283,84.031933
7,"fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",104,190470.901034,6739.451402
8,"fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",37,13075.351870,398.065559
9,"fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",11,43407.079719,1611.299605


In [7]:
# Combine the two dataframes
combined_df = pd.merge(ml_features_df, performance_metrics_df, on="hamiltonian_id")

In [8]:
combined_df

,hamiltonian_id,num_orbitals,num_electrons,num_spin_orbitals,two_S,orbsym,isym,total_num_states,log10_hilbert_space_size,tbt_matrices_calc_time_s,...,tbt_mean,tbt_std,tbt_max,tbt_min,tbt_coeff_variation,tbt_inv_coeff_variation,tbt_calc_time_s,coarse_bond_dimension,total_loops_cpu_time_sec,total_loops_wall_time_sec
0,"fcidump.9_mo_n2-_{'Mo'_ 'def2-TZVP', 'default'...",33,46,66,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.066117e+16,16.609180,0.000129,...,-0.000181,0.008872,0.267286,-0.419490,-48.890471,-0.020454,0.004729,19,11575.997945,371.488722
1,fcidump.10_fecp2+_s0.5_def2-tzvp,46,57,92,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.970221e+25,25.472789,0.000156,...,-0.000110,0.006374,0.199710,-0.327631,-57.896224,-0.017272,0.625121,13,33469.678334,1172.941756
2,fcidump.11_fecp2_s0_def2-tzvp,46,58,92,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,1.792375e+25,25.253429,0.000148,...,-0.000105,0.006246,0.196475,-0.318972,-59.572611,-0.016786,0.452869,8,14890.202160,538.292357
3,"fcidump.12_mo_n2_{'Mo'_ 'def2-TZVP', 'default'...",33,45,66,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,8.906733e+16,16.949718,0.000099,...,-0.000208,0.009116,0.267021,-0.459342,-43.897176,-0.022781,0.004100,170,52218.561128,1580.383785
4,"fcidump.20_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",32,50,64,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.785570e+13,13.679934,0.000068,...,-0.000267,0.010970,0.185329,-0.572850,-41.013888,-0.024382,0.003799,9,4534.212040,166.208680
5,"fcidump.21_rc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000170,...,-0.000084,0.005300,0.280269,-0.456726,-63.056685,-0.015859,0.525287,77,164849.615598,5863.330374
6,"fcidump.24_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",27,44,54,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,2.393071e+10,10.378956,0.000081,...,-0.000382,0.013949,0.151781,-0.593228,-36.478773,-0.027413,0.001649,9,2490.699283,84.031933
7,"fcidump.25_ts_1over2_{'Mo'_ 'def2-SVP', 'I'_ '...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000184,...,-0.000083,0.005330,0.267025,-0.452306,-64.516594,-0.015500,0.465496,104,190470.901034,6739.451402
8,"fcidump.26_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",32,50,64,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,4.785570e+13,13.679934,0.000063,...,-0.000281,0.010940,0.168178,-0.564789,-38.961989,-0.025666,0.004385,37,13075.351870,398.065559
9,"fcidump.27_pc_{'Mo'_ 'def2-SVP', 'I'_ 'def2-SV...",51,74,102,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,9.533095e+24,24.979234,0.000185,...,-0.000081,0.005307,0.248773,-0.445743,-65.850445,-0.015186,0.669791,11,43407.079719,1611.299605


In [9]:
# Save the combined dataframe to csv
combined_df.to_csv("combined_df.csv", index=False)
